Predicting Stock Price

In [1]:
import pandas as pd
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

In [2]:
#Read the data and change the date column to datetime objects
df = pd.read_csv("sphist.csv")
df['Date'] = pd.to_datetime(df['Date'])
#sort by date from oldest to newest
df = df.sort_values("Date", ascending=True)

In [3]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


In [4]:
#Add features that could be helpful for machine learning
df['5 Days Open'] = df['Open'].rolling(center=False, window=5).mean()
df['5 Days High'] = df['High'].rolling(center=False, window=5).mean()
df['5 Days Low'] = df['Low'].rolling(center=False, window=5).mean()
df['5 Days Volume'] = df['Volume'].rolling(center=False, window=5).mean()
df['Year'] = df['Date'].apply(lambda x: x.year)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,1950
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,1950
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,1950
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,1950
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,16.9,16.9,16.9,2046000.0,1950


In [5]:
#Adding Day of week column and set it to categorical
df['DOW'] = df['Date'].apply(lambda x: x.weekday())
dow_df = pd.get_dummies(df['DOW'])
df = pd.concat([df, dow_df], axis=1)
df = df.drop(['DOW'], axis=1)

In [6]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year,0,1,2,3,4
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,1950,0,1,0,0,0
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,1950,0,0,1,0,0
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,1950,0,0,0,1,0
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,1950,0,0,0,0,1
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,16.9,16.9,16.9,2046000.0,1950,1,0,0,0,0


In [7]:
#Shift the columns by one
df['5 Days Open'] = df['5 Days Open'].shift(1)
df['5 Days High'] = df['5 Days High'].shift(1)
df['5 Days Low'] = df['5 Days Low'].shift(1)
df['5 Days Volume'] = df['5 Days Volume'].shift(1)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year,0,1,2,3,4
16589,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66,NaN,NaN,NaN,NaN,1950,0,1,0,0,0
16588,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85,NaN,NaN,NaN,NaN,1950,0,0,1,0,0
16587,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93,NaN,NaN,NaN,NaN,1950,0,0,0,1,0
16586,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98,NaN,NaN,NaN,NaN,1950,0,0,0,0,1
16585,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08,NaN,NaN,NaN,NaN,1950,1,0,0,0,0


In [8]:
df = df[df['Date'] >= datetime(year=1951, month=1, day=3)]
df.dropna(axis=0)

,Date,Open,High,Low,Close,Volume,Adj Close,5 Days Open,5 Days High,5 Days Low,5 Days Volume,Year,0,1,2,3,4
16339,1951-01-03,20.690001,20.690001,20.690001,20.690001,3.370000e+06,20.690001,20.360000,20.360000,20.360000,3.126000e+06,1951,0,0,1,0,0
16338,1951-01-04,20.870001,20.870001,20.870001,20.870001,3.390000e+06,20.870001,20.514000,20.514000,20.514000,3.268000e+06,1951,0,0,0,1,0
16337,1951-01-05,20.870001,20.870001,20.870001,20.870001,3.390000e+06,20.870001,20.628000,20.628000,20.628000,3.358000e+06,1951,0,0,0,0,1
16336,1951-01-08,21.000000,21.000000,21.000000,21.000000,2.780000e+06,21.000000,20.726001,20.726001,20.726001,3.324000e+06,1951,1,0,0,0,0
16335,1951-01-09,21.120001,21.120001,21.120001,21.120001,3.800000e+06,21.120001,20.840001,20.840001,20.840001,3.192000e+06,1951,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883,2088.579980,2093.966064,2080.503955,3.207544e+09,2015,0,1,0,0,0
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010,2087.283984,2095.518066,2080.811963,3.232372e+09,2015,0,0,1,0,0
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117,2090.741992,2097.548047,2082.175977,3.245514e+09,2015,0,0,0,1,0
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941,2089.023975,2095.948047,2073.385962,3.536224e+09,2015,0,0,0,0,1


In [9]:
train = df[df['Date'] < datetime(year=2013, month=1, day=1)]
test = df[df['Date'] >= datetime(year=2013, month=1, day=1)]


In [10]:
features = ['5 Days Open', '5 Days Volume', '5 Days High', '5 Days Low', 'Year', 0, 1, 2, 3, 4]

In [11]:
lr = LinearRegression()
lr.fit(train[features], train['Close'])
predictions = lr.predict(test[features])

mae = mean_absolute_error(test['Close'] ,predictions)

print(df.tail(1))
print(mae)

        Date         Open         High          Low        Close  \
0 2015-12-07  2090.419922  2090.419922  2066.780029  2077.070068   

         Volume    Adj Close  5 Days Open  5 Days High   5 Days Low  \
0  4.043820e+09  2077.070068  2081.507959  2096.058057  2066.807983   

   5 Days Volume  Year  0  1  2  3  4  
0   4.085838e+09  2015  1  0  0  0  0  
14.600316723381313
